# Efficiency vs quality tradeoffs

Author: Monique Monteiro - moniquelouise@gmail.com



In [31]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [1]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-10"

## Libraries installation

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=e395c7562144b17daeb1f64c54ee50a00f8261d9c96eaa0a6c9a3120a65d1222
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

!pip install pyserini -q
!pip install faiss-cpu -q

In [4]:
!pip install evaluate -q
!pip install trectools -q

In [5]:
!pip install transformers -q

## Passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-05-06 21:33:47--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.121, 18.155.68.38, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683668029&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSo

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-10/trec-covid/corpus.jsonl already exists; do you wish to overwrite (y or n)? ^C


## Queries vs passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-05-06 21:34:54--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  1.09MB/s    in 0.9s    

2023-05-06 21:34:55 (1.09 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

## Queries download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-05-06 21:34:56--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683668096&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-10/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? ^C


## Pipeline 1: Pyserini BM25 + Reranking with cross-encoder/ms-marco-MiniLM-L-6-v2

### 1st step: Pre-indexing TREC-COVID corpus

Here, we need to measure indexing cost in dollars per hour.

In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
device

device(type='cuda')

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

embedder = SentenceTransformer(model_name, device=device)



Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cross-encoder_ms-marco-MiniLM-L-6-v2 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import jsonlines

passage_ids = []
passage_texts = []
id_to_passage = dict()

#Maps each id to its position in the corpus
id_to_index = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    id_to_index[id] = len(passage_ids)
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_passage[id] = text

In [ ]:
%time corpus_embeddings = embedder.encode(passage_texts, convert_to_tensor=True)

CPU times: user 6min 29s, sys: 10.1 s, total: 6min 39s
Wall time: 3min


In [ ]:
corpus_embeddings.shape

torch.Size([171332, 384])

In [ ]:
%%writefile gpu_usage.sh
#! /bin/bash
#comment: run for 10 seconds, change it as per your use
end=$((SECONDS+3600))

while [ $SECONDS -lt $end ]; do
    nvidia-smi --format=csv --query-gpu=power.draw,utilization.gpu,memory.used,memory.free,fan.speed,temperature.gpu >> gpu.log
    #comment: or use below command and comment above using #
    #nvidia-smi dmon -i 0 -s mu -d 1 -o TD >> gpu.log
done

Overwriting gpu_usage.sh


In [ ]:
%%bash --bg

bash gpu_usage.sh

In [ ]:
corpus_embeddings = embedder.encode(passage_texts, convert_to_tensor=True) #2m41s

In [ ]:
corpus_embeddings.shape

torch.Size([171332, 384])

### Pipeline - Searching with Pyserini BM25 + Reranking

In [8]:
from pyserini.search.lucene import LuceneSearcher
import numpy as np
import json

In [9]:
def search_with_bm25(query,k = 1000):
  searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
  hits = searcher.search(query, k)
  return hits

In [10]:
import jsonlines

query_ids = []
query_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)
     

In [ ]:
query_texts[0]

'what is the origin of COVID-19'

Generates the first request to download pre-indexed corpus.

In [ ]:
hits = search_with_bm25(query_texts[0])

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:20, 11.1MB/s]                           


In [ ]:
len(hits)

1000

In [ ]:
jsondoc = json.loads(hits[0].raw)
jsondoc

{'_id': 'dv9m19yk',
 'title': '[What is the origin of SARS-CoV-2?]',
 'text': 'Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.',
 'metadata': {'url': 'https://www.ncbi.nlm.nih.gov/pubmed/32412715/',
  'pubmed_id': '32412715'}}

In [ ]:
id_to_passage['dv9m19yk']

'[What is the origin of SARS-CoV-2?] Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.'

In [ ]:
id_to_index['dv9m19yk']

81848

In [ ]:
passage_ids[81848]

'dv9m19yk'

In [ ]:
passage_texts[81848]

'[What is the origin of SARS-CoV-2?] Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.'

In [ ]:
corpus_embeddings[81848].shape

torch.Size([384])

In [ ]:
def search(query, top_k=1000):
  #1st step: searches with BM25
  bm25_hits = search_with_bm25(query, k=top_k)

  ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
  indices = [id_to_index[id] for id in ids]

  #Finds corpus embeddings
  passages_embeddings = [corpus_embeddings[i] for i in indices]

  query_embedding = embedder.encode(query, convert_to_tensor=True)

  hits = util.semantic_search(query_embedding, passages_embeddings, top_k=top_k)
  hits = hits[0]      #Get the hits for the first query
  results = dict()
  for hit in hits:
    assert hit['corpus_id'] < 1000
    results[ids[hit['corpus_id']]] = hit['score']

  return results

In [ ]:
%time search(query_texts[0])

In [ ]:
id_to_passage['gh07kf93']

'What are the economic implications of COVID-19 '

In [ ]:
id_to_passage['ghuwz4w7']

'What the Diamond Princess taught the world about covid-19 '

In [ ]:
id_to_passage['x33dcjov']

'What the COVID-19 Crisis Is Telling Humanity '

In [ ]:
id_to_passage['xefpg0zo']

'What’s Important: Facing Fear in the Time of COVID-19 '

### Measuring NDCG@10

In [11]:

import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [ ]:
len(qrel["q0"])

66336

In [ ]:
from collections import defaultdict
from tqdm import tqdm

def search_all(top_k=1000):
  run = defaultdict(list)

  for id, query in tqdm(zip(query_ids, query_texts)):
    results = search(query)
    run["query"] += [id] * top_k
    run["docid"] += results.keys()
    run["score"] += results.values()
    run["q0"] += ["q0"] * top_k
    run["rank"] += list(range(1,top_k+1))
    run["system"] += [model_name] * top_k

  return run

In [ ]:
run = search_all()

50it [00:07,  6.52it/s]


In [ ]:
len(run["system"])

50000

In [12]:

from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

Evaluates BM25 NDCG@10.

In [ ]:
def search_all_bm25(top_k=1000):
  run = defaultdict(list)

  for id, query in tqdm(zip(query_ids, query_texts)):
    bm25_hits = search_with_bm25(query)
    ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
    run["query"] += [id] * top_k
    run["docid"] += ids
    run["score"] += [1] * top_k
    run["q0"] += ["q0"] * top_k
    run["rank"] += list(range(1,top_k+1))
    run["system"] += [model_name] * top_k

  return run


In [ ]:
run_bm25 = search_all_bm25()

50it [00:05,  9.08it/s]


In [ ]:
eval_ndcg10(run_bm25)

0.5946917010118077

In [ ]:
eval_ndcg10(run)

0.2754485142056899

Conclusion: horrible quality for off-the-shelf usage.  It requiries fine tuning to be used on TREC-COVID.

## Pipeline 2: SPLADE

In [13]:
from transformers import BertTokenizer, BertForMaskedLM, DistilBertTokenizer, DistilBertForMaskedLM
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [14]:
model_name_1 = 'naver/splade_v2_distil' 
model_name_2 = 'naver/splade-cocondenser-selfdistil'
model_name_3 = 'naver/splade-cocondenser-ensembledistil' 

#tokenizer_1 = DistilBertTokenizer.from_pretrained(model_name_1)
#model_1 = DistilBertForMaskedLM.from_pretrained(model_name_1)

#tokenizer_2 = BertTokenizer.from_pretrained(model_name_2)
#model_2 = BertForMaskedLM.from_pretrained(model_name_2)

tokenizer_3 = AutoTokenizer.from_pretrained(model_name_3)
model_3 = AutoModelForMaskedLM.from_pretrained(model_name_3)

In [15]:
model = model_3
tokenizer = tokenizer_3

In [16]:
def vectorize_to_sparse_batch(batch, model=model, remove_special_tokens=True):
  model.to(device)
  model.eval()
  
  with torch.no_grad():
    output = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))

  w_ij = output.logits.to(device)

  #Kudos Leandro Carísio
  mask_tokens_validos = batch['attention_mask'].to(device)
  mask = mask_tokens_validos.unsqueeze(-1).expand(w_ij.size())
  
  wj = torch.max(torch.log(1 + relu(w_ij*mask)), dim=1)[0]

  #According to ChatGPT: In PyTorch, the to_sparse() method is used to convert a 
  #dense tensor into a sparse tensor.  A dense tensor is a tensor that contains 
  #all elements, including those with zero values. In contrast, a sparse tensor 
  #is a tensor that only stores the non-zero elements, along with their indices. 
  #Sparse tensors are useful when working with large tensors with mostly zero 
  #values, as they can save memory and computational resources.
  return wj.to_sparse()

In [17]:
import torch
from torch.nn.functional import relu
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def vectorize_to_sparse(text, tokenizer=tokenizer, model=model, remove_special_tokens=False):
  # Kudos to Marcos Piau
  with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=True):
    tokenized_text = tokenizer(text, max_length=max_length, truncation=True, 
                              return_tensors='pt', 
                              return_special_tokens_mask=True).to(device)
    
    model.to(device)
    model.eval()
    
    with torch.no_grad():
      output = model(tokenized_text['input_ids'].to(device), 
                     attention_mask=tokenized_text['attention_mask'].to(device))

    w_ij = output.logits[0,:]

    mask_tokens_validos = 1 - tokenized_text['special_tokens_mask'].to(device)

    #Kudos Leandro Carísio
    #1) Removes all dimenstions of size 1 (here, it's the first one)
    #2) Adds a 1 dimension to the end (equivalent to transpose, I think)
    #3) Replicates (expands) up to the vocabulary size -> 5 X 30k 
    if remove_special_tokens:
      mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
    else:
      mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(w_ij.size())
      mask = torch.ones(mask.size()).to(device)

    wj = torch.max(torch.log(1 + relu(w_ij*mask)), dim=0)[0]

  #According to ChatGPT: In PyTorch, the to_sparse() method is used to convert a 
  #dense tensor into a sparse tensor.  A dense tensor is a tensor that contains 
  #all elements, including those with zero values. In contrast, a sparse tensor 
  #is a tensor that only stores the non-zero elements, along with their indices. 
  #Sparse tensors are useful when working with large tensors with mostly zero 
  #values, as they can save memory and computational resources.
  return wj.to_sparse()

In [18]:
import jsonlines

passage_ids = []
passage_texts = []
id_to_text = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_text[id] = text

#Sorts the passages by length
passage_indices = sorted(range(len(passage_texts)), 
                         key=lambda k: len(passage_texts[k]))
passage_texts = sorted(passage_texts, key=lambda k: len(k))
passage_ids = sorted(passage_ids, key=lambda k: len(id_to_text[k]))

In [19]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts['input_ids'])
    
    def __getitem__(self, idx):
        #return self.tokenized_texts[idx]
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'special_tokens_mask': self.tokenized_texts['special_tokens_mask'][idx]
        }

In [20]:
from transformers import BatchEncoding

def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

In [21]:
max_length=256
batch_size = 32

In [ ]:
import time

start = time.time()
test_passages_tokenized = tokenizer(passage_texts, max_length=max_length, 
                                   truncation=True, padding=True, 
                                   return_special_tokens_mask=True)
dataset_passages_test = Dataset(test_passages_tokenized)
dataloader_passages_test = data.DataLoader(dataset_passages_test, 
                                          batch_size=batch_size, shuffle=False, 
                                          collate_fn=collate_fn)
end = time.time()
print("Time spent on tokenization = ", end - start)

Time spent on tokenization =  30.496001720428467


In [ ]:
len(dataset_passages_test)

171332

In [ ]:
import torch
from tqdm import tqdm
import time

start = time.time()

docs_matrix = None

# kudos Marcos Piau
with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
  for doc_batch in tqdm(dataloader_passages_test, mininterval=0.5, desc='Test', 
                                          disable=False):
    doc_vector = vectorize_to_sparse_batch(doc_batch, model=model, sum_or_max=False)

    if docs_matrix is None:
      docs_matrix = doc_vector
    else:
      docs_matrix = torch.cat((docs_matrix, doc_vector), dim=0)

end = time.time()
print("Time spent on matrix building = ", end - start)

Test: 100%|██████████| 5355/5355 [38:01<00:00,  2.35it/s]

Time spent on matrix building =  2291.792766571045


In [ ]:
import pickle

with open(f"{main_dir}/trec-covid/docs_matrix.pickle", "wb") as f:
  pickle.dump(docs_matrix, f)

In [ ]:
!nvidia-smi

### Inverted Index

In [22]:
import array
import pandas as pd
from collections import defaultdict
from collections import Counter
import pickle
import os

index_path = f"{main_dir}/index.pickle"

def load_or_build_inverted_index():
  if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      index = pickle.load(f)
  else:
    print("Building inverted index...")
    inverted_index = dict()
    idx = 0

    def process(doc_id, idx):
      assert passage_ids[idx] == doc_id
      doc_vec = docs_matrix[idx]
      doc_vec = doc_vec.coalesce()
      indices = doc_vec.indices()[0]
      values = doc_vec.values()

      for token_id, wj in zip(indices, values):
        token_id = token_id.item()
        wj = wj.item()
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["docs"].append(idx)
        inverted_index.setdefault(
            token_id, {"docs":array.array("L", []), 
                       "wj":array.array("f", [])})["wj"].append(wj)

    for i in tqdm(range(docs_matrix.shape[0])):
      process(passage_ids[i], i)
      
    index = {"inverted_index": inverted_index}

    with open(index_path, "wb") as f:
      pickle.dump(index, f)

  return index

In [23]:
start = time.time()
index = load_or_build_inverted_index()
end = time.time()
print("Time spent to build inverted index = ", end - start)

Loading index...
Time spent to build inverted index =  0.8949294090270996


In [24]:
inverted_index = index["inverted_index"]

In [57]:
len(inverted_index)

26050

In [38]:
import math
import torch.nn.functional as F
import time

def search_by_query_vector_in_inverted_index(query_vec, k, ids=None):
  query_vec = query_vec.coalesce()
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  doc_ids = []
  indices = query_vec.indices()[0]
  values = query_vec.values()

  for token_id, wj in zip(indices, values):
    token_id = token_id.item()
    wj = wj.item()
    
    if token_id in inverted_index:
      doc_ids = inverted_index[token_id]["docs"]
      wjs = inverted_index[token_id]["wj"]

      for idx, doc_wj in zip(doc_ids, wjs):
        if ids is not None and passage_ids[idx] in ids:
          doc_scores[passage_ids[idx]] += wj * doc_wj
        elif ids is None:
          doc_scores[passage_ids[idx]] += wj * doc_wj
        
  doc_scores = dict(sorted(doc_scores.items(), key=lambda x:x[1], 
                           reverse=True)[:k])
          
  return doc_scores
     

In [26]:
k = 1000

In [27]:
len(query_ids)

50

In [ ]:

from collections import defaultdict
import time

run_splade_inv = defaultdict(list)

start = time.time()

for i, query in zip(query_ids, query_texts):
  query_vec = vectorize_to_sparse(query, tokenizer=tokenizer, model=model, 
                        sum_or_max=False)
  doc_scores = search_by_query_vector_in_inverted_index(query_vec, k)
  run_splade_inv["query"] += [i] * k
  run_splade_inv["docid"] += doc_scores.keys()
  run_splade_inv["score"] += doc_scores.values()
  run_splade_inv["q0"] += ["q0"] * k
  run_splade_inv["rank"] += list(range(1,k+1))
  run_splade_inv["system"] += ["splade"] * k

end = time.time()
print("Time spent = ", end - start)
print("Time spent by query = ", (end - start)/len(query_ids))

Time spent =  65.70718932151794
Time spent by query =  1.314143786430359


In [ ]:
eval_ndcg10(run_splade_inv)

0.7268509214947312

## 3rd Pipelie: Pyserini BM25 + Splade Reranking

Here, in the SAPLDE step, as we already have a filered list with 1000 documents, we use directly the matrix of documents instead of inverted index.  So we assume BM25 has already used an inverted index.

In [28]:
k=1000

In [ ]:
docs_matrix

tensor(indices=tensor([[     0,      0,      0,  ..., 171331, 171331, 171331],
                       [  1037,   1041,   1996,  ...,  29514,  29566,  29610]]),
       values=tensor([0.4171, 0.0068, 0.1628,  ..., 0.1091, 0.2448, 0.0680]),
       device='cuda:0', size=(171332, 30522), nnz=27975739,
       layout=torch.sparse_coo)

In [35]:
import time

def search_bm25_splade(query, top_k=1000):
  #1st step: searches with BM25
  bm25_hits = search_with_bm25(query, k=top_k)
  
  ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
  ids = set(ids)

  query_embedding = vectorize_to_sparse(query, tokenizer=tokenizer, model=model)
  
  doc_scores = search_by_query_vector_in_inverted_index(query_embedding, k, ids=ids)
  
  return doc_scores

In [44]:
from collections import defaultdict

run_splade_matrix = defaultdict(list)

start = time.time()

for i, query in zip(query_ids, query_texts):
  doc_scores = search_bm25_splade(query, k)
  n = len(doc_scores)
  run_splade_matrix["query"] += [i] * n
  run_splade_matrix["docid"] += doc_scores.keys()
  run_splade_matrix["score"] += doc_scores.values()
  run_splade_matrix["q0"] += ["q0"] * n
  run_splade_matrix["rank"] += list(range(1,n+1))
  run_splade_matrix["system"] += ["bm25_splade"] * n

end = time.time()
print("Time spent = ", end - start)
print("Time spent by query = ", (end - start)/len(query_ids))

Time spent =  26.299543142318726
Time spent by query =  0.5259908628463745


In [41]:
len(run_splade_matrix["query"])

50000

In [42]:
len(run_splade_matrix["docid"])

49946

In [43]:
len(run_splade_matrix["score"])

49946

In [45]:
eval_ndcg10(run_splade_matrix)

0.7419530766303556